In [1]:
#setup autogpt with a read and write file tool
from langchain.agents import Tool
from langchain_community.tools.file_management.read import ReadFileTool
from langchain_community.tools.file_management.write import WriteFileTool
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [2]:
#setup memory used for agent's intermediate steps
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


In [3]:
#Define your embedding model
embeddings_model = OpenAIEmbeddings()
#Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query,index,InMemoryDocstore({}),{})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [4]:
#setup model and autogpt, initialize everything and use ChatOpenAI model
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [5]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role = "Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
)
#set verbose to be true
agent.chain.verbose= True

In [6]:
#Run an example
#Here we will make it write a weather report for SF
agent.run(["write a weather report for SF today"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. write a weather report for SF today


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {

'I have completed the task of checking the weather and writing a weather report for San Francisco today. I will now finish my objectives.'

In [7]:
#Chat History Memory
from langchain_community.chat_message_histories import FileChatMessageHistory

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)
